**Set environment**

In [1]:
### set env
source ../config_duke.sh -v
source ../config_load_module_bedtools.sh

In [2]:
ls ${FD_RES}/source

cCREs      starrseq_220510     syn22348180  tmp
remap2022  starrseq_peak_macs  syn26435687


**Check data**

In [3]:
ls ${FD_RES}/source/starrseq_peak_macs

KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed


In [4]:
ls ${FD_RES}/source/cCREs

ENCFF971AHO_ENCFF847JMY_ENCFF779QTH_ENCFF405AYC.7group.bed


In [5]:
ls ${FD_RES}/source/remap2022

remap2022_K562_all_macs2_hg38_v1_0.bed.gz
remap2022_K562_crm_macs2_hg38_v1_0.bed.gz
remap2022_K562_nr_macs2_hg38_v1_0.bed.gz


In [6]:
zcat ${FD_RES}/source/remap2022/remap2022_K562_crm_macs2_hg38_v1_0.bed.gz | head

chr1	9829	10459	MNT,NCOA1,ZNF24,NFATC3,EHMT2,TCF12,SMARCE1,AFF1,EP400,MCM3,HDAC1,E2F8,NR2C1,PRDM10,NUFIP1,LEF1,MCM5,ZBTB40,NONO,TRIM24,SNIP1,ZNF184,ZBTB5	148	.	10101	10102	204,120,188
chr1	136455	137130	NCOR1,HDAC1,MTA3,L3MBTL2,DPF2,ZBTB33,HES1,HDGF,ATF3,CTBP1,SMARCA5,BRD9,ZNF639,EGR1,NFIC,HDAC2,SKIL,SMARCE1,NR2C1,ARID1B,TRIM24,ZBTB40,ZBTB2,ZNF592	33	.	136762	136763	1,115,178
chr1	181203	181789	CTCFL,SETDB1	14	.	181462	181463	2,158,115
chr1	267963	268138	CTCF,RAD21	84	.	268034	268035	202,145,97
chr1	586110	586241	RAD21	14	.	586192	586193	222,143,5
chr1	605324	605608	MYC,CTCFL,MAX,E2F6,RBFOX2	11	.	605440	605441	251,175,228
chr1	609289	609559	CTCF,ZBTB33	15	.	609392	609393	202,145,97
chr1	610470	610796	ZBTB33,CTCF	26	.	610653	610654	1,115,178
chr1	611372	611806	ZBTB33	2	.	611520	611521	2,158,115
chr1	629069	629435	TARDBP	7	.	629240	629241	213,94,0

gzip: stdout: Broken pipe


## Annotation with CCRE

In [7]:
sbatch -p ${NODE} \
    --mem 20G \
    -o ${FD_LOG}/annotation_astarr_input_ccre.out.txt \
    -e ${FD_LOG}/annotation_astarr_input_ccre.err.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh

### set input and output
FD_BED=${FD_RES}/source/starrseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/source/cCREs
FN_ANN=ENCFF971AHO_ENCFF847JMY_ENCFF779QTH_ENCFF405AYC.7group.bed
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/KS91_K562_ASTARRseq/annotation
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.union_narrowPeak.ccre_all.bed.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo
echo "Input  file 1: " ${FP_BED}
echo "Input  file 2: " ${FP_ANN}
echo "Output file:   " ${FP_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FP_BED}
cat  ${FP_BED} | head -n 3 
echo
echo ${FP_ANN}
cat  ${FP_ANN} | head -n 3 
echo

### annotation using intersect
bedtools intersect -a ${FP_BED} -b ${FP_ANN} -wo -F 1.0 | gzip -c > ${FP_OUT}

### print end message
echo "Show the first few lines of the output file"
echo ${FP_OUT}
zcat ${FP_OUT} | head -n 3
echo

timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28340053


## Annotate with ReMap

In [8]:
sbatch -p ${NODE} \
    --mem 20G \
    -o ${FD_LOG}/annotation_astarr_input_remap.out.txt \
    -e ${FD_LOG}/annotation_astarr_input_remap.err.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh

### set input and output
FD_BED=${FD_RES}/source/starrseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/source/remap2022
FN_ANN=remap2022_K562_nr_macs2_hg38_v1_0.bed.gz
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/KS91_K562_ASTARRseq/annotation
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.union_narrowPeak.remap2022_nr.bed.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo
echo "Input  file 1: " ${FP_BED}
echo "Input  file 2: " ${FP_ANN}
echo "Output file:   " ${FP_OUT}
echo
echo "Show the first few lines of the input file"
echo  ${FP_BED}
cat   ${FP_BED} | head -n 3 
echo
echo  ${FP_ANN}
zcat  ${FP_ANN} | head -n 3 
echo

### annotation using intersect
bedtools intersect -a ${FP_BED} -b ${FP_ANN} -wo -F 1.0 | gzip -c > ${FP_OUT}

### print end message
echo "Show the first few lines of the output file"
echo ${FP_OUT}
zcat ${FP_OUT} | head -n 3
echo

timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28340054


## Check results

In [9]:
cat ${FD_LOG}/annotation_astarr_input_ccre.out.txt

Slurm Array Index: 
Time Stamp:         05-28-22+15:45:36

Input  file 1:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/starrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
Input  file 2:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/cCREs/ENCFF971AHO_ENCFF847JMY_ENCFF779QTH_ENCFF405AYC.7group.bed
Output file:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/annotation/KS91_K562_hg38_ASTARRseq_Input.all_reps.union_narrowPeak.ccre_all.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/starrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/cCREs/ENCFF971AHO_ENCFF847JMY_ENCFF779QTH_ENCFF405AYC.7group.bed
chr1	181251	181601	EH38E1310153	0	.	181251	181601	6,218

In [10]:
cat ${FD_LOG}/annotation_astarr_input_ccre.err.txt

In [11]:
cat ${FD_LOG}/annotation_astarr_input_remap.out.txt

Slurm Array Index: 
Time Stamp:         05-28-22+15:45:36

Input  file 1:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/starrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
Input  file 2:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/remap2022/remap2022_K562_nr_macs2_hg38_v1_0.bed.gz
Output file:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/annotation/KS91_K562_hg38_ASTARRseq_Input.all_reps.union_narrowPeak.remap2022_nr.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/starrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/remap2022/remap2022_K562_nr_macs2_hg38_v1_0.bed.gz
chr1	9899	10349	EP400:K-562	1	.	10089	10090	102,132,184
chr1	9901	10232	NCOA1:K

In [13]:
cat ${FD_LOG}/annotation_astarr_input_remap.err.txt